In [1]:
from genres import collect_genre_features

In [2]:
genre_playlists = {
    'bluegrass':'https://www.youtube.com/playlist?list=PLKUA473MWUv2mddNMPh-MJkgTR5AjTpl3',
    'blues':'https://www.youtube.com/playlist?list=PLxvodScTx2RuB7Yh_d21kD0HDZvDXWBbY',
    'folk':'https://www.youtube.com/playlist?list=PLGBuKfnErZlBLNzS_JlDAeiH5aW26rvHc',
    'classic_rock':'https://www.youtube.com/playlist?list=PLNxOe-buLm6cz8UQ-hyG1nm3RTNBUBv3K'
}
collect_genre_features(genre_playlists, data_fp='data/extra_genre_features.json')

ERROR: This video is no longer available because the YouTube account associated with this video has been terminated.
Sorry about that.
ERROR: This video contains content from BBC Studios, who has blocked it on copyright grounds.
Sorry about that.
ERROR: This video contains content from UMG, who has blocked it in your country on copyright grounds.
Sorry about that.
ERROR: This video contains content from UMG. It is not available.
Sorry about that.
ERROR: This video contains content from UMG, who has blocked it in your country on copyright grounds.
Sorry about that.
ERROR: This video contains content from UMG, who has blocked it in your country on copyright grounds.
Sorry about that.
ERROR: This video contains content from UMG, who has blocked it in your country on copyright grounds.
Sorry about that.
ERROR: This video contains content from UMG. It is not available in your country.
Sorry about that.
ERROR: This video contains content from WMG, who has blocked it in your country on copyri

In [3]:
import pandas as pd

In [4]:
df = pd.read_json('data/extra_genre_features.json', lines=True)

In [7]:
df.groupby('genre').song.nunique()

genre
bluegrass       87
blues           92
classic_rock    80
folk            74
Name: song, dtype: int64

In [8]:
import pickle

In [19]:
with open('data/extra_genre_features.pkl', 'wb') as f:
    pickle.dump(df.to_json(), f)

In [20]:
df_original = pd.read_json('data/genre_features.json', lines=True)

In [21]:
with open('data/genre_features.pkl', 'wb') as f:
    pickle.dump(df_original.to_json(), f)